Sudoku Solver by Evan Freeman
Basic Technique, Then Brute Force


Input must be a string of 81 characters
Blank cells may be filled with anything for the input.



For example:
    .94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8
    
Which solves to:
    794582136268931745315476982689715324432869571157243869821657493943128657576394218


Here's the coordinates of every cell in the grid:

(0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8)

(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8)

(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8)

(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8)

(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8)

(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8)

(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8)

(7,0) (7,1) (7,2) (7,3) (7,4) (7,5) (7,6) (7,7) (7,8)

(8,0) (8,1) (8,2) (8,3) (8,4) (8,5) (8,6) (8,7) (8,8)


Big picture:
    1) Start with naked / hidden singles techniques. This is enough for many puzzle.
        -Use the "little numbers" technique, where each cell has a "bag" of possible numbers that gets updated.
        -We will need to store these numbers in a list, so we can iterate through them as we 
    2) When you get stuck, finish off with brute force, but only considering 

To do:
    1) Accept more input types
    2) Check for multiple solutions
    3) Implement sudoku strategies to speed up solve time:
        -Could still finish off with brute force, even if I just put in a few basic strategies
        -Do the 'little numbers' technique, where I keep track of the possibilites associated with each cell
    4) Make it display as it goes?
        -That means showing MILLIONS of iterations. Would have to animate SUPER FAST.
        -Or collect all those images and make a gif of it, after the fact.
            - Pre-complile??? I don't know what I'm talking about
    5) Just how brute force is my algorithm? I think it's slightly faster than just populating every blank with a guess,
        checking the whole puzzle, then updating the whole puzzle and checking again (or even just checking the parts effected
        by the update).
        -After all, my algorithm cuts off certian possibility spaces as it goes
        -Still pretty brute force
    6) YES, I SHOULD HAVE MADE THE BLANKS OBJECTS, MAYBE EVEN ASSOCIATED THEM WITH THE CELL OBJECT SOMEHOW. SUE ME!!!
        -also should have named them "blanks" instead of "b"
        -Actually, maybe just a dict would have been better
    7) Add another strategy.
    8) Maybe add doubles and triples
        
        
Thoughts:
    Brute force time to complete depends on a few things:
        1) Luck. Whether a given blank cell will end up being 1 or 9 makes a big difference, as we start at 1.
        2) Contradiction depth. How long, on average, we can randomly fill cells before creating a contradiction?

In [1]:
import time
import pdb

In [3]:
#Here is my original, pretty brute force, method
def solve_bf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, in the format of ['.', i, j]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                #so keep track of the cell we are going to fill, and it's coordinates
                b.append([sudoku.cells[i][j], i, j])

    #Initialize some variables
    i = 0    
    count = 0 

    #This is the engine that drives the solution 
    #In each scenario, we update both the list of blanks, and the sudoku grid itself
    #Keep going until our index hits the length of blanks (Which is to say, we're one step beyond)
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with 1
        if b[i][0] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
            b[i][0] = '1'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at 9. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == '9':
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some number 1-8 already plugged in. So we step forward by one.
        else:
            b[i][0] = str(int(b[i][0]) + 1)
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1

    #Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

In [5]:
#Here is my less brute force solver, which only brute forces over the possiblities for each cell, not all of 1-9

def solve_lbf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the less brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                b.append([sudoku.cells[i][j], i, j, real_poss])
    
    
    
    #Step 2: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility
    
    i = 0
    count = 0
    
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if b[i][0] == '.':
            b[i][0] = b[i][3][0]
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == b[i][3][-1]:
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            b[i][0] = b[i][3][b[i][3].index(b[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

In [4]:
#Here is my updated solver
#It uses only hidden single and naked single, then finishes with brute force if necessary
#Ok, working on adding doubles and triples

def solve(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the mostly not brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():
        pdb.set_trace()
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        c1_blanks = [blank for blank in blanks if blank[2] == 0]
        
        r1_blanks = [blank for blank in blanks if blank[1] == 0]
                
        b1_blanks = [blank for blank in blanks if blank[1] == 0]
        
        
        return False
            
            
            
                
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    nt_count = 0
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = naked_triple()
                    update_blanks
                    progress = progress or prog4
                    nt_count += prog4
            
        
        
        
        
            
    
    sudoku.display()
    print(f'We solved {ns_count} cells with naked singles.')
    print(f'We solved {hs_count} cells with hidden singles.')
    print(f'We helped {nd_count} times with naked doubles.')
    print(f'We helped {nt_count} times with naked triples.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'Brute force: {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('-'*200)
    print('\n')
    return solution

In [ ]:
# I've implemented some particular puzzles for it to solve.
# You can uncomment the lines below to make it ask for an input,or just plug in your own puzzle below.

# puzzle = input('Please input your sudoku puzzle as a string of 81 characters. ')      
# solve(puzzle)

In [ ]:
#Random Sudoku
#This requires a HARD strat, like "simple coloring" or "single chaining"
#It basically is a sort of guess and check.
#Does use naked pairs
#Eww

solve_bf('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')
solve_lbf('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')
solve('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')

Here is the brute force solution result:

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 2329276 loops.


--- This program took 38.575459003448486 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 1219898 loops.


--- This program took 19.

(Pdb)  n


> <ipython-input-4-0136957f1801>(167)naked_double()
-> blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]


(Pdb)  n


> <ipython-input-4-0136957f1801>(170)naked_double()
-> blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]


...

> <ipython-input-4-0136957f1801>(256)solve()
-> prog3 = naked_double()


(Pdb)  


> <ipython-input-4-0136957f1801>(163)naked_double()
-> for blank in blanks:


(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  
(Pdb)  stop


*** NameError: name 'stop' is not defined


In [ ]:
# Easy Puzzle 5,220,548,762 from Web Sudoku

solve_bf('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')
solve_lbf('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')
solve('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')

In [ ]:
#Some easy one, forgot where this came from

solve_bf('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')
solve_lbf('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')
solve('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')

In [ ]:
#Kindof Hard 1

solve_bf('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')
solve_lbf('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')
solve('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')

In [ ]:
#More of a medium

solve_bf('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')
solve_lbf('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')
solve('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')

In [ ]:
#Greg's Evil Puzzle

solve_bf('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')
solve_lbf('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')
solve('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')

In [ ]:
#Easiest Possible Sudoku, from sudoku wiki
#Only requires naked singles

solve_bf('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')
solve_lbf('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')
solve('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')

In [ ]:
#Very Hard Benchmark 1

solve_bf('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
solve_lbf('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
solve('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')

In [ ]:
#Very Hard Benchmark 1

solve('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')

In [ ]:
#Here's a repository of 95 hard puzzle, a good benchmark

puzzles = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......52...6.........7.13...........4..8..6......5...........418.........3..2...87.....6.....8.3.4.7.................5.4.7.3..2.....1.6.......2.....5.....8.6......1....48.3............71.2.......7.5....6....2..8.............1.76...3.....4......5........14....3....2...7..........9...3.6.1.............8.2.....1.4....5.6.....7.8.........52..8.4......3...9...5.1...6..2..7........3.....6...1..........7.4.......3.6.2.5.........3.4..........43...8....1....2........7..5..27...........81...6......524.........7.1..............8.2...3.....6...9.5.....1.6.3...........897........6.2.5.........4.3..........43...8....1....2........7..5..27...........81...6......923.........8.1...........1.7.4...........658.........6.5.2...4.....7.....9.....6..3.2....5.....1..........7.26............543.........8.15........4.2........7...6.5.1.9.1...9..539....7....4.8...7.......5.8.817.5.3.....5.2............76..8.....5...987.4..5...1..7......2...48....9.1.....6..2.....3..6..2.......9.7.......5..3.6.7...........518.........1.4.5...7.....6.....2......2.....4.....8.3.....5.....1.....3.8.7.4..............2.3.1...........958.........5.6...7.....8.2...4.......6..3.2....4.....1..........7.26............543.........8.15........4.2........7......3..9....2....1.5.9..............1.2.8.4.6.8.5...2..75......4.1..6..3.....4.6.45.....3....8.1....9...........5..9.2..7.....8.........1..4..........7.2...6..8...237....68...6.59.9.....7......4.97.3.7.96..2.........5..47.........2....8.........84...3....3.....9....157479...8........7..514.....2...9.6...2.5....4......9..56.98.1....2......6.............3.2.5..84.........6.........4.8.93..5...........1....247..58..............1.4.....2...9528.9.4....9...1.........3.3....75..685..2...4.....8.5.3..........7......2.....6.....5.4......1.......6.3.7.5..2.....1.9.......2.3......63.....58.......15....9.3....7........1....8.879..26......6.7...6..7..41.....7.9.4...72..8.........7..1..6.3.......5.6..4..2.........8..53...7.7.2....464.....3.....8.2......7........1...8734.......6........5...6........1.4...82.............71.2.8........4.3...7...6..5....2..3..9........6...7.....8....4......5....6..3.2....4.....8..........7.26............543.........8.15........8.2........7...47.8...1............6..7..6....357......5....1..6....28..4.....9.1...4.....2.69.......8.17..2........5.6......7...5..1....3...8.......5......2..4..8....6...3....38.6.......9.......2..3.51......5....3..1..6....4......17.5..8.......9.......7.32...5...........5.697.....2...48.2...25.1...3..8..3.........4.7..13.5..9..2...31...2.......3.5.62..9.68...3...5..........64.8.2..47..9....3.....1.....6...17.43.....8..4....3......1........2...5...4.69..1..8..2...........3.9....6....5.....2.......8.9.1...6.5...2......6....3.1.7.5.........9..4...3...5....2...7...3.8.2..7....44.....5.8.3..........7......2.....6.....5.8......1.......6.3.7.5..2.....1.8......1.....3.8.6.4..............2.3.1...........958.........5.6...7.....8.2...4.......1....6.8..64..........4...7....9.6...7.4..5..5...7.1...5....32.3....8...4........249.6...3.3....2..8.......5.....6......2......1..4.82..9.5..7....4.....1.7...3......8....9.873...4.6..7.......85..97...........43..75.......3....3...145.4....2..1...5.1....9....8...6.......4.1..........7..9........3.8.....1.5...2..4.....36..........8.16..2........7.5......6...2..1....3...8.......2......7..3..8....5...4.....476...5.8.3.....2.....9......8.5..6...1.....6.24......78...51...6....4..9...4..7.....7.95.....1...86..2.....2..73..85......6...3..49..3.5...41724.................4.5.....8...9..3..76.2.....146..........9..7.....36....1..4.5..6......3..71..2...834.........7..5...........4.1.8..........27...3.....2.6.5....5.....8........1....9.....3.....9...7.....5.6..65..4.....3......28......3..75.6..6...........12.3.8.26.39......6....19.....7.......4..9.5....2....85.....3..2..9..4....762.........42.3.8....8..7...........1...6.5.7...4......3....1............82.5....6...1.......6..3.2....1.....5..........7.26............843.........8.15........8.2........7..1.....9...64..1.7..7..4.......3.....3.89..5....7....2.....6.7.9.....4.1....129.3..........9......84.623...5....6...453...1...6...9...7....1.....4.5..2....3.8....9.2....5938..5..46.94..6...8..2.3.....6..8.73.7..2.........4.38..7....6..........59.4..5...25.6..1..31......8.7...9...4..26......147....7.......2...3..8.6.4.....9....52.....9...3..4......7...1.....4..8..453..6...1...87.2........8....32.4..8..1.53..2.9...24.3..5...9..........1.827...7.........981.............64....91.2.5.43.1....786...7..8.1.8..2....9........24...1......9..5...6.8..........5.9.......93.4....5...11......7..6.....8......4.....9.1.3.....596.2..8..62..7..7......3.5.7.2...47.2....8....1....3....9.2.....5...6..81..5.....4.....7....3.4...9...1.4..27.8........94.....9...53....5.7..8.4..1..463...........7.8.8..7.....7......28.5.26.....2......6....41.....78....1......7....37.....6..412....1..74..5..8.5..7......39..1.....3.8.6.4..............2.3.1...........758.........7.5...6.....8.2...4.......2....1.9..1..3.7..9..8...2.......85..6.4.........7...3.2.3...6....5.....1.9...2.5..7..8.....6.2.3...3......9.1..5..6.....1.....7.9....2........4.83..4...26....51....36....85.......9.4..8........68.........17..9..45...1.5...6.4....9..2.....3...34.6.......7.......2..8.57......5....7..1..2....4......36.2..1.......9.......7.82......4.18..2........6.7......8...6..4....3...1.......6......2..5..1....7...3.....4..5..67...1...4....2.....1..8..3........2...6...........4..5.3.....8..2...............4...2..4..1.7..5..9...3..7....4..6....6..1..8...2....1..85.9...6.....8...38..7....4.5....6............3.97...8....43..5....2.9....6......2...6...7.71..83.2.8...4.5....7..3............1..85...6.....2......4....3.26............417............7..8...6...5...2...3.61.1...7..2..8..534.2..9.......2......58...6.3.4...1..........8.16..2........7.5......6...2..1....3...8.......2......7..4..8....5...3.....2..........6....3.74.8.........3..2.8..4..1.6..5.........1.78.5....9..........4..52..68.......7.2.......6....48..9..2..41......1.....8..61..38.....9...63..6..1.9....1.78.5....9..........4..2..........6....3.74.8.........3..2.8..4..1.6..5.....1.......3.6.3..7...7...5..121.7...9...7........8.1..2....8.64....9.2..6....4.....4...7.1....19.46.5.....1......7....2..2.3....847..6....14...8.6.2....3..6...9..........8.17..2........5.6......7...5..1....3...8.......5......2..3..8....6...4....963......1....8......2.5....4.8......1....7......3..257......3...9.2.4.7......9..15.3......7..4.2....4.72.....8.........9..1.8.1..8.79......38...........6....7423..........5724...98....947...9..3...5..9..12...3.1.9...6....25....56.....7......6....75....1..2.....4...3...5.....3.2...8...1.......6.....1..48.2........7........6.....7.3.4.8.................5.4.8.7..2.....1.3.......2.....5.....7.9......1........6...4..6.3....1..4..5.77.....8.5...8.....6.8....9...2.9....4....32....97..1...32.....58..3.....9.428...1...4...39...6...5.....1.....2...67.8.....4....95....6....5.3.......6.7..5.8....1636..2.......4.1.......3...567....2.8..4.7.......2..5...5.3.7.4.1.........3.......5.8.3.61....8..5.9.6..1........4...6...6927....2...9....5..8..18......9.......78....4.....64....9......53..2.6.........138..5....9.714...........72.6.1....51...82.8...13..4.........37.9..1.....238..5.4..9.........79....658.....4......12............96.7...3..5....2.8...3..19..8..3.6.....4....473...2.3.......6..8.9.83.5........2...8.7.9..5........6..4.......1...1...4.22..7..8.9.5..9....1.....6.....3.8.....8.4...9514.......3....2..........4.8...6..77..15..6......2.......7...17..3...9.8..7......2.89.6...13..6....9..5.824.....891..........3...8.......7....51..............36...2..4....7...........6.13..452...........8..'

n=81

puzzle_list = [puzzles[i:i+n] for i in range(0, len(puzzles), n)]


for puzzle in puzzle_list:
    solve(puzzle)



In [4]:
a = 5
b = 4